# Demo Notebook:
## DeSurv

In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.


In [2]:
# import os
# from pathlib import Path
# import sys

# node_type = os.getenv('BB_CPU')
# venv_dir = '/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/virtual_envs/SurvivEHRbenchmarking/'
# venv_path = venv_dir + f'cpu-{node_type}'
# venv_site_pkgs = Path(venv_path) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
# if venv_site_pkgs.exists():
#     sys.path.insert(0, str(venv_site_pkgs))
#     print(f"Added path '{venv_site_pkgs}' at start of search paths.")
# else:
#     print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

# %load_ext autoreload
# %autoreload 2

In [3]:
# import pytorch_lightning
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
from dataclasses import dataclass
import logging
from FastEHR.dataloader import FoundationalDataModule
import pickle
from tqdm import tqdm

from pycox.datasets import support
from pycox.evaluation import EvalSurv
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from torch.utils.data import TensorDataset, DataLoader

from CPRD.src.modules.head_layers.survival.desurv import ODESurvSingle
from CPRD.src.modules.head_layers.survival.desurv import ODESurvMultiple

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

Using device: cuda.


# Load data

In [4]:
def get_dataloaders(dataset, competing_risk, sample_size=None, seed=None):

    match dataset.lower():
        case "pycox":
            df_train = support.read_df()
            df_test = df_train.sample(frac=0.2)
            df_train = df_train.drop(df_test.index)
            df_val = df_train.sample(frac=0.2)
            df_train = df_train.drop(df_val.index)
            
            cols_standardize = ['x0', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13']
            cols_leave = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
            
            standardize = [([col], StandardScaler()) for col in cols_standardize]
            leave = [(col, None) for col in cols_leave]
            
            x_mapper = DataFrameMapper(standardize + leave)
            
            x_train = x_mapper.fit_transform(df_train).astype('float32')
            x_val = x_mapper.transform(df_val).astype('float32')
            x_test = x_mapper.transform(df_test).astype('float32')
            
            get_target = lambda df: (df['duration'].values, df['event'].values)
            y_train = get_target(df_train)
            y_val = get_target(df_val)
            y_test = get_target(df_test)
            
            t_train, e_train = y_train
            t_val, e_val = y_val
            t_test, e_test = y_test
            
            t_train_max = np.amax(t_train)
            t_train = t_train / t_train_max
            t_val = t_val / t_train_max
            t_test = t_test / t_train_max
            
    
        case "hypertension" | "cvd":
    
            # Training samples
            if sample_size is not None:
                save_path =  f"/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_{dataset}/" + f"benchmark_data/N={sample_size}_seed{seed}.pickle" 
            else:
                save_path = f"/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_{dataset}/" + "benchmark_data/all.pickle"
                
            with open(save_path, "rb") as handle:
                print(f"Loading training dataset from {save_path}")
                data_train = pickle.load(handle)
                print(f"{data_train['X_train'].shape}")
            
            # display(data["X_train"].head())
            # display(data["y_train"])
            # print(data.keys())
            
            data = {}
            data["X_train"] = data_train["X_train"]
            data["y_train"] = data_train["y_train"]
    
            # Test and validation samples
    
            save_path = f"/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_{dataset}/" + "benchmark_data/all.pickle"
            with open(save_path, "rb") as handle:
                print(f"Loading validation/test datasets from {save_path}")
                data_val_test = pickle.load(handle)
                
            data["X_val"] = data_val_test["X_val"]
            data["y_val"] = data_val_test["y_val"]
            data["X_test"] = data_val_test["X_test"]
            data["y_test"] = data_val_test["y_test"]
    
            # Convert to correct formats
            x_train = data["X_train"].to_numpy(dtype=np.float32)
            x_val = data["X_val"].to_numpy(dtype=np.float32)
            x_test = data["X_test"].to_numpy(dtype=np.float32)
            
            t_train = np.asarray([i[1] for i in data["y_train"]])
            t_val = np.asarray([i[1] for i in data["y_val"]])        
            t_test = np.asarray([i[1] for i in data["y_test"]])
    
            if competing_risk is False:
                e_train = np.asarray([0 if i[0] == 0 else 1 for i in data["y_train"]])
                e_val = np.asarray([0 if i[0] == 0 else 1 for i in data["y_val"]])
                e_test = np.asarray([0 if i[0] == 0 else 1 for i in data["y_test"]])
            else:
                e_train = np.asarray([i[0] for i in data["y_train"]])
                e_val = np.asarray([i[0] for i in data["y_val"]])
                e_test = np.asarray([i[0] for i in data["y_test"]])

    # display(x_train.shape)
    # display(type(x_train))
    # display(type(x_train[0,0]))
    # display(e_train.shape)
    # display(type(e_train))
    # display(type(e_train[0]))
    # display(t_train.shape)
    # display(type(t_train))
    # display(type(t_train[0]))
    # print(np.mean(e_train))
    # print(np.mean(t_train))
    # print(np.std(t_train))
    # print(np.mean(x_train))
    # print(t_train.min())
    # print(t_train.max())
    # print(np.unique(e_test, return_counts=True))
    
    
    batch_size = 256
    dataset_train = TensorDataset(*[torch.tensor(u,dtype=dtype_) for u, dtype_ in [(x_train,torch.float32),
                                                                                   (t_train,torch.float32),
                                                                                   (e_train,torch.long)]])
    data_loader_train = DataLoader(dataset_train, batch_size=batch_size, pin_memory=True, shuffle=True, drop_last=True)
    
    dataset_val = TensorDataset(*[torch.tensor(u,dtype=dtype_) for u, dtype_ in [(x_val,torch.float32),
                                                                                   (t_val,torch.float32),
                                                                                   (e_val,torch.long)]])
    data_loader_val = DataLoader(dataset_val, batch_size=batch_size, pin_memory=True, shuffle=True)
    
    dataset_test = TensorDataset(*[torch.tensor(u,dtype=dtype_) for u, dtype_ in [(x_test,torch.float32),
                                                                                   (t_test,torch.float32),
                                                                                   (e_test,torch.long)]])
    data_loader_test = DataLoader(dataset_test, batch_size=batch_size, pin_memory=True, shuffle=True)

    return data_loader_train, data_loader_val, data_loader_test




# Example dataloader function usage

In [6]:
data_loader_train, data_loader_val, data_loader_test = get_dataloaders("Hypertension", False, sample_size=None, seed=1)

Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
(572096, 279)
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


In [7]:
for batch in data_loader_test:
    x_train = batch[0]
    print(batch[0].shape)
    print(torch.mean(batch[1]))
    print(batch[2])
    break

torch.Size([256, 279])
tensor(1.4163)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])


# Train model
```
Hypertension

SR
[0.768850848388541]
[0.08287342061953376]
[0.2629869386534056]

CVD

SR
[0.6660374450461991]
[0.03355264167471281]
[0.14207857084831382]

CR
[0.6622136902603167]
[0.033540062894645686]
[0.142287892974895]
```

In [10]:
dataset = "Hypertension" 
competing_risk = False
# sample_sizes = [int(np.exp(_log_n)) for _log_n in np.linspace(np.log(3000), np.log(500000), 10)]      # [3000, 12500, 30000, 60000, 100000]: # 600, 1200, 
sample_sizes = [2999, 5296, 9351, 16509, 29148, 51461, 90856, 160407, 283203, 500000, None]
sample_sizes = [None]
# sample_sizes = [500000]
# sample_sizes = sample_sizes[-1:]

# training = True
lr = 1e-3
xdim = x_train.shape[1]


# the time grid which we generate over
t_eval = np.linspace(0, 1, 1000) 
# the time grid which we calculate scores over
time_grid = np.linspace(start=0, stop=1 , num=300)

In [ ]:
model_names, all_ctd, all_ibs, all_inbll = [], [], [], []

for sample_size in sample_sizes:

    seeds = [1,2,3,4,5]

    for seed in seeds:
        # Load dataset
        data_loader_train, data_loader_val, data_loader_test = get_dataloaders(dataset, competing_risk, sample_size=sample_size, seed=seed)
    
        # Initialise model
        model_name = f"DeSurv-{dataset}-Ns{sample_size}-seed{seed}"
        if competing_risk is False:
            model = ODESurvSingle(xdim, [32, 32], device=device)
        else:
            model = ODESurvMultiple(xdim, [32, 32], num_risks=5)
        print(f"\n\n{model_name} with {sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters")

        # Load or train model
        torch.manual_seed(seed)
        try:
            state_dict = torch.load(model_name + "tst_model")
            model.load_state_dict(state_dict)
            print(f"Loaded previously trained model")
        except:
            print(f"Training")
            model.optimize(data_loader_train, n_epochs=20, logging_freq=1, data_loader_val=data_loader_val, max_wait=2)
            print("finished training")
            torch.save(model.state_dict(), model_name + "tst_model")
            model.eval()
    
            # state_dict = torch.load(model_name + "tst_model")
            # model.load_state_dict(state_dict)
           
        
        # argsortttest = np.argsort(t_test)
        # t_test = t_test[argsortttest]
        # e_test = e_test[argsortttest]
        # x_test = x_test[argsortttest,:]
        
        print(f"Testing")    
        model.eval()
        with torch.no_grad():
    
            ctd = []
            ibs = []
            inbll = []
            for batch in tqdm(data_loader_test, total=(len(data_loader_test)), desc="Testing"):
    
                x_test = batch[0].numpy()
                t_test = batch[1].numpy()
                e_test = batch[2].numpy()
    
                # The normalised grid over which to predict
                t_test_grid = torch.tensor(np.concatenate([t_eval] * x_test.shape[0], 0), dtype=torch.float32)
                x_test_grid = torch.tensor(x_test, dtype=torch.float32).repeat_interleave(t_eval.size, 0)
                
                pred_bsz = 51200
                pred = []
                for x_test_batched, t_test_batched in zip(torch.split(x_test_grid, pred_bsz), torch.split(t_test_grid, pred_bsz)):
                    
                    if competing_risk is False:
                        pred_ = model.predict(x_test_batched, t_test_batched)          # shape: (x_test.batched.shape[0],)
                    else:
                        pred_, pi_  = model.predict(x_test_batched, t_test_batched)    # shape: (x_test.batched.shape[0], num_outcomes)
                    pred.append(pred_)
                        
                pred = torch.concat(pred)
            
                pred = pred.reshape((x_test.shape[0], t_eval.size, -1)).cpu().detach().numpy()
                preds = [pred[:, :, _i] for _i in range(pred.shape[-1])]
                # print([_.shape for _ in preds])
        
                # Merge (additively) each outcome risk curve into a single CDF, and update label for if outcome occurred or not
                cdf = np.zeros_like(preds[0])
                lbls = np.zeros_like(e_test)     
                for _outcome_token in np.unique(e_test)[1:]:
                    # print(f"{_outcome_token} of {np.unique(e_test)[1:]} included from {len(preds)} surv CDFs")
                    # print(_outcome_token)
                    cdf += preds[_outcome_token - 1] 
                    lbls += (e_test == _outcome_token)
                
                surv = pd.DataFrame(np.transpose((1 - cdf.reshape((x_test.shape[0],t_eval.size)))), index=t_eval)
                # print(surv)
    
                # Evaluate surv curve with unscaled index with unscaled test times to event 
                ev = EvalSurv(surv, t_test, lbls, censor_surv='km')
                # try:
                    # Same treatment as in SurvivEHR
                ctd.append(ev.concordance_td())
                ibs.append(ev.integrated_brier_score(time_grid))
                inbll.append(ev.integrated_nbll(time_grid))
                # print(f"{ctd} {ibs}, {inbll}")
                # except:
                #     pass
                
            ctd = np.mean(ctd)
            ibs = np.mean(ibs)
            inbll = np.mean(inbll)
    
            print(f"{model_name}:".ljust(20) + f"N={sample_size}.".ljust(15) + f"Ctd: {ctd}. IBS: {ibs}. INBLL: {inbll}")

        model_names.append(model_name)
        all_ctd.append(ctd)
        all_ibs.append(ibs)
        all_inbll.append(inbll)


Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
(572096, 279)
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


DeSurv-Hypertension-NsNone-seed1 with 10081 parameters
Training
	Epoch:  0. Total loss:   262342.64
best_epoch: 0
	Epoch:  0. Total val loss:    14993.43
	Epoch:  1. Total loss:   247637.81
best_epoch: 1
	Epoch:  1. Total val loss:    14772.03
	Epoch:  2. Total loss:   245857.33
	Epoch:  2. Total val loss:    14809.54
	Epoch:  3. Total loss:   244750.78
best_epoch: 3
	Epoch:  3. Total val loss:    14764.87
	Epoch:  4. Total loss:   243993.23
best_epoch: 4
	Epoch:  4. Total val loss:    14751.96
	Epoch:  5. Total loss:   243276.68
best_epoch: 5
	Epoch:  5. Total val loss:    14706.59
	Epoch:  6. Total loss:   242696.47
	Epoch:  6. Total val loss:    14747

Testing: 100%|██████████| 140/140 [00:14<00:00,  9.93it/s]


DeSurv-Hypertension-NsNone-seed1:N=None.        Ctd: 0.7708035555642572. IBS: 0.08267526462773596. INBLL: 0.2625306123346124
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
(572096, 279)
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


DeSurv-Hypertension-NsNone-seed2 with 10081 parameters
Training
	Epoch:  0. Total loss:   264250.41
best_epoch: 0
	Epoch:  0. Total val loss:    14995.57
	Epoch:  1. Total loss:   249065.96
best_epoch: 1
	Epoch:  1. Total val loss:    14806.92
	Epoch:  2. Total loss:   246848.68
	Epoch:  2. Total val loss:    14929.38
	Epoch:  3. Total loss:   245412.42
best_epoch: 3
	Epoch:  3. Total val loss:    14699.10
	Epoch:  4. Total loss:   244466.25
	Epoch:  4. Total val loss:    14812.86
	Epoch:  5. Total loss:   243831.36
	Epoch:  5. Tot

Testing: 100%|██████████| 140/140 [00:14<00:00,  9.88it/s]


DeSurv-Hypertension-NsNone-seed2:N=None.        Ctd: 0.7695751200897563. IBS: 0.08273968181756607. INBLL: 0.262755230788484
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
(572096, 279)
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


DeSurv-Hypertension-NsNone-seed3 with 10081 parameters
Training
	Epoch:  0. Total loss:   263525.15
best_epoch: 0
	Epoch:  0. Total val loss:    15057.56
	Epoch:  1. Total loss:   248587.60
best_epoch: 1
	Epoch:  1. Total val loss:    14861.83
	Epoch:  2. Total loss:   246639.46
best_epoch: 2
	Epoch:  2. Total val loss:    14745.99
	Epoch:  3. Total loss:   245216.52
	Epoch:  3. Total val loss:    14906.11
	Epoch:  4. Total loss:   244196.80
best_epoch: 4
	Epoch:  4. Total val loss:    14715.75
	Epoch:  5. Total loss:   243366.60
	E

Testing: 100%|██████████| 140/140 [00:14<00:00,  9.82it/s]


DeSurv-Hypertension-NsNone-seed3:N=None.        Ctd: 0.7724139601735452. IBS: 0.08270569036836782. INBLL: 0.2625610527660121
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
(572096, 279)
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


DeSurv-Hypertension-NsNone-seed4 with 10081 parameters
Training
	Epoch:  0. Total loss:   265191.16
best_epoch: 0
	Epoch:  0. Total val loss:    15200.15
	Epoch:  1. Total loss:   249490.30
best_epoch: 1
	Epoch:  1. Total val loss:    14855.25
	Epoch:  2. Total loss:   246943.33
best_epoch: 2
	Epoch:  2. Total val loss:    14765.10
	Epoch:  3. Total loss:   245495.67
best_epoch: 3
	Epoch:  3. Total val loss:    14728.11
	Epoch:  4. Total loss:   244464.41
best_epoch: 4
	Epoch:  4. Total val loss:    14718.48
	Epoch:  5. Total loss:

Testing: 100%|██████████| 140/140 [00:14<00:00,  9.63it/s]


DeSurv-Hypertension-NsNone-seed4:N=None.        Ctd: 0.7729993169568111. IBS: 0.08248514899737762. INBLL: 0.2621711782943788
Loading training dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle
(572096, 279)
Loading validation/test datasets from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/benchmark_data/all.pickle


DeSurv-Hypertension-NsNone-seed5 with 10081 parameters
Training
	Epoch:  0. Total loss:   266318.23
best_epoch: 0
	Epoch:  0. Total val loss:    14920.48
	Epoch:  1. Total loss:   247918.10
best_epoch: 1
	Epoch:  1. Total val loss:    14816.83
	Epoch:  2. Total loss:   245815.22
best_epoch: 2
	Epoch:  2. Total val loss:    14786.59
	Epoch:  3. Total loss:   244699.37
best_epoch: 3
	Epoch:  3. Total val loss:    14723.82
	Epoch:  4. Total loss:   243835.01
best_epoch: 4
	Epoch:  4. Total val loss:    14681.30
	Epoch:  5. Total loss:

In [ ]:
print(model_names)
print(all_ctd)
print(all_ibs)
print(all_inbll)

In [ ]:
print(surv.head())
print(surv.shape)
print(preds[0].shape)

In [ ]:
# sample_sizes[-1] = 572096
print(sample_sizes)
print(all_ctd)
print(all_ibs)
print(all_inbll)

# Output across different setups

Hypertension Single Risk

Hypertension Competing Risk

In [ ]:
# Hypertension_cr:N=2999.        Ctd: 0.6815094745489083. IBS: 0.09055914251601428. INBLL: 0.31029164842950396
# Hypertension_cr:N=5296.        Ctd: 0.7124953181062986. IBS: 0.08950243267368668. INBLL: 0.2998138831024105
# Hypertension_cr:N=9351.        Ctd: 0.7274689831170883. IBS: 0.08788381776138605. INBLL: 0.29219335002924224
# Hypertension_cr:N=16509.       Ctd: 0.730682386913791. IBS: 0.08713170539614512. INBLL: 0.2915679214433027
# Hypertension_cr:N=29148.       Ctd: 0.7389774246331557. IBS: 0.08589961516053798. INBLL: 0.2797340688171351
# Hypertension_cr:N=51461.       Ctd: 0.7411341306621856. IBS: 0.08509218669879787. INBLL: 0.27232015638634294
# Hypertension_cr:N=90856.       Ctd: 0.7515646895266107. IBS: 0.08443103008855017. INBLL: 0.27128357563201616
# Hypertension_cr:N=160407.      Ctd: 0.7600540925292503. IBS: 0.08451878180772691. INBLL: 0.2692630746813842
# Hypertension_cr:N=283203.      Ctd: 0.7620305063798369. IBS: 0.08404512042043363. INBLL: 0.2666228576617026
# Hypertension_cr:N=500000.      Ctd: 0.761360566628242. IBS: 0.08371696663855675. INBLL: 0.26584980102348854

Cardiovascular disease Single Risk

Cardiovascular disease Competing Risk

In [ ]:
# CVD_cr:          N=2999.        Ctd: 0.5625189849883911. IBS: 0.03435665860871859. INBLL: 0.14990771301641295
# CVD_cr:          N=5296.        Ctd: 0.5869489534497859. IBS: 0.03411933684032459. INBLL: 0.14823684700350284
# CVD_cr:          N=9351.        Ctd: 0.5993921885610577. IBS: 0.03442101009915632. INBLL: 0.14955773914208445
# CVD_cr:          N=16509.       Ctd: 0.6101382510977712. IBS: 0.03387871579676817. INBLL: 0.14660058710195034
# CVD_cr:          N=29148.       Ctd: 0.6200633894187596. IBS: 0.033806085396407184. INBLL: 0.1458525721893707
# CVD_cr:          N=51461.       Ctd: 0.624818164536866. IBS: 0.033786555455045275. INBLL: 0.14535719048483275
# CVD_cr:          N=90856.       Ctd: 0.6355840601226502. IBS: 0.03364728847608413. INBLL: 0.14414393636167722
# CVD_cr:          N=160407.      Ctd: 0.6469820170508909. IBS: 0.033556625283347644. INBLL: 0.14308219809140393
# CVD_cr:          N=283203.      Ctd: 0.6557505764681258. IBS: 0.033524955828030355. INBLL: 0.1423563785027365
# CVD_cr:          N=500000.      Ctd: 0.6582288879213832. IBS: 0.033605688713984706. INBLL: 0.14273257727425628
